<a href="https://colab.research.google.com/github/srepasup/ML---Book-Recommendation-Engine-using-KNN/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-08 02:18:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   124MB/s    in 0.2s    

2025-05-08 02:18:37 (124 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [8]:
# Load and clean dataset
books = pd.read_csv("BX-Books.csv", encoding='latin-1', sep=';', on_bad_lines='skip') # Specify the encoding as 'latin-1' and delimiter as ';', skip bad lines
ratings = pd.read_csv("BX-Book-Ratings.csv", encoding='latin-1', sep=';', on_bad_lines='skip') # Specify the encoding as 'latin-1' and delimiter as ';', skip bad lines

<ipython-input-8-3de804364c27>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv", encoding='latin-1', sep=';', on_bad_lines='skip') # Specify the encoding as 'latin-1' and delimiter as ';', skip bad lines


In [10]:
# Remove users and books below the threshold
user_counts = ratings["User-ID"].value_counts() # Changed "user_id" to "User-ID"
book_counts = ratings["ISBN"].value_counts() # Changed "isbn" to "ISBN"

filtered_users = user_counts[user_counts >= 200].index
filtered_books = book_counts[book_counts >= 100].index

filtered_ratings = ratings[ratings["User-ID"].isin(filtered_users) & ratings["ISBN"].isin(filtered_books)] # Changed "user_id" to "User-ID" and "isbn" to "ISBN"

In [12]:
# Merge with books dataset
combined = filtered_ratings.merge(books, on="ISBN").drop_duplicates() # Changed 'isbn' to 'ISBN' to match the column name in the 'books' DataFrame

In [15]:
# Build the Nearest Neighbors model
model = NearestNeighbors(metric="cosine", algorithm="brute")
# Changed 'isbn' to 'ISBN' and 'rating' to 'Book-Rating' to match the actual column names in the 'combined' DataFrame
book_features = combined.pivot(index="ISBN", columns="User-ID", values="Book-Rating").fillna(0)
model.fit(book_features)

NearestNeighbors(algorithm='brute', metric='cosine')

In [26]:
# Define the recommendation function
def get_recommends(book_title):
    # Access the 'Book-Title' column instead of 'title'
    book_index = combined[combined["Book-Title"] == book_title].index[0]
    distances, indices = model.kneighbors(book_features.iloc[book_index].values.reshape(1, -1), n_neighbors=6)

    # Access the 'Book-Title' column for recommendations
    recommendations = [(combined.iloc[idx]["Book-Title"], dist) for idx, dist in zip(indices[0][1:], distances[0][1:])]
    return [book_title, recommendations]

In [27]:
# Example usage:
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

['The Queen of the Damned (Vampire Chronicles (Paperback))', [('The Firm', np.float64(0.695843135126752)), ("White Oleander : A Novel (Oprah's Book Club)", np.float64(0.727385010920766)), ('Jurassic Park', np.float64(0.7598300195870005)), ('The Witching Hour (Lives of the Mayfair Witches)', np.float64(0.7752522227432805)), ('Misery', np.float64(0.7833967509268956))]]
